# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [89]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [90]:
csv_path = os.path.join("..", "WeatherPy", "Output", "output.csv")
original_weather_df = pd.read_csv(csv_path)
original_weather_df[["Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Date"]] = original_weather_df[["Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Date"]].astype(float)
original_weather_df.dropna(axis=0,how='all', inplace=True)#drop all NaN items from data frame
original_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Puerto Ayora,-0.74,-0.74,72.00,90.0,27.0,3.00,EC,1.608036e+09
1,Qandala,11.47,11.47,77.04,58.0,10.0,10.71,SO,1.608036e+09
2,Albany,42.60,42.60,28.00,49.0,0.0,1.99,US,1.608035e+09
3,Barrow,71.29,71.29,-2.20,70.0,1.0,16.11,US,1.608035e+09
4,Busselton,-33.65,-33.65,63.00,63.0,0.0,5.01,AU,1.608036e+09
...,...,...,...,...,...,...,...,...,...
627,Lata,40.16,40.16,61.00,74.0,29.0,5.01,PT,1.608036e+09
628,Port-Cartier,50.03,50.03,16.00,66.0,100.0,10.29,CA,1.608036e+09
629,Pouébo,-20.40,-20.40,76.46,77.0,2.0,13.87,NC,1.608036e+09
630,Yuzhno-Sukhokumsk,44.66,44.66,46.27,75.0,100.0,16.37,RU,1.608036e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [91]:
#configure gmaps using my Google API key
gmaps.configure(api_key=g_key)

coordinates = original_weather_df[['Lat', 'Lng']]
humidity = original_weather_df["Humidity"]
# Customize the size of the figure
figure_layout = {
    'width': '533px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)  # Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)

heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 0.05)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [112]:
good_weather_df = original_weather_df.copy()

good_weather_df = good_weather_df[(good_weather_df['Max Temp'] > 70) & (good_weather_df['Max Temp'] < 80)] #only show cities with a max temperature between 70 and 80 degrees
good_weather_df = good_weather_df[good_weather_df['Wind Speed'] > 10] #and only show cities where the wind speed is <10 mph
good_weather_df = good_weather_df[good_weather_df['Cloudiness'] == 0] #and only show cities where there's no clouds
good_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
55,Cape Town,-33.93,-33.93,73.99,46.0,0.0,26.40,ZA,1.608035e+09
58,Poum,-20.23,-20.23,79.27,73.0,0.0,18.01,NC,1.608036e+09
68,Lüderitz,-26.65,-26.65,78.80,47.0,0.0,29.97,NaN,1.608036e+09
79,Port Elizabeth,-33.92,-33.92,77.00,50.0,0.0,17.22,ZA,1.608035e+09
88,Carnarvon,-24.87,-24.87,75.20,83.0,0.0,17.22,AU,1.608036e+09
133,Saldanha,-33.01,-33.01,77.00,41.0,0.0,19.46,ZA,1.608036e+09
154,Sur,22.57,22.57,71.51,49.0,0.0,12.37,OM,1.608036e+09
169,Nouadhibou,20.93,20.93,73.40,46.0,0.0,19.46,MR,1.608036e+09
232,Jijiga,9.35,9.35,76.50,17.0,0.0,13.22,ET,1.608036e+09
287,Dakar,14.69,14.69,78.80,69.0,0.0,11.41,SN,1.608035e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [93]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
